In [164]:
import random
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn.model_selection import train_test_split

# set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# if you ever use GPU, these help determinism
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False

In [165]:
# converts 'MM:SS' strings to float minutes
def convert_mp(mp):
    if isinstance(mp, str):
        if ':' in mp:
            try:
                mins, secs = mp.split(':')
                return int(mins) + int(secs) / 60
            except:
                return None
        else:
            try:
                return float(mp)
            except:
                return None
    elif isinstance(mp, (int, float)):
        return float(mp)
    else:
        return None


In [166]:
# import data
#data = pd.read_csv('../data-collection/bbref_players_games_simple/g/gilgesh01_Shai_Gilgeous-Alexander_last3.csv')
data = pd.read_csv('../data-collection/bbref_players_games_simple/gilgesh01_Shai_Gilgeous-Alexander_last3_with_opp_stats.csv')
# make sure games are ordered oldest -> newest
if 'Date' in data.columns:
    data = data.sort_values('Date')

# is_home: 1 if home, 0 if away
data['is_home'] = (data['Unnamed: 5'] != '@').astype(float)
# result_win: 1 if team won, 0 if lost
data['result_win'] = data['Result'].str.startswith('W').astype(float)

# features (previous game)
features = [
    'MP', 'FGA', '3PA', 'FTA',
    'FG%', '3P%', '2P%', 'eFG%', 'FT%',
    'TRB', 'AST', 'TOV',
    'GmSc', '+/-', 'PTS', 'is_home', 'result_win',
    'OppOffRtg', 'OppDefRtg', 'OppNetRtg', 'OppPace'
]

# convert MP to float minutes
data['MP'] = data['MP'].apply(convert_mp)

# convert numeric columns
for col in features + ['PTS']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# create TARGET from *current* game points (before shifting)
current_ppg = 32.4
data['above_ppg'] = (data['PTS'] > current_ppg).astype(float)
target = ['above_ppg']

# now shift features so they refer to the *previous* game
data[features] = data[features].shift(1)

# drop first row and rows with NaNs
data = data.dropna(subset=features + target)

# numpy arrays
X_np = data[features].values
y_np = data[target].values

# chronological split: last 20 games as test
n_test = 20
X_train_np, X_test_np = X_np[:-n_test], X_np[-n_test:]
y_train_np, y_test_np = y_np[:-n_test], y_np[-n_test:]

# to torch
X_train = torch.tensor(X_train_np, dtype=torch.float32)
y_train = torch.tensor(y_train_np, dtype=torch.float32)
X_test = torch.tensor(X_test_np, dtype=torch.float32)
y_test = torch.tensor(y_test_np, dtype=torch.float32)

In [167]:
# define model, loss function, and optimizer
model = nn.Sequential(
    nn.Linear(X_train.shape[1], 1),
    nn.Sigmoid()
)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)

epochs = 1000
lambda_penalty = .8  # tune this (start small like 0.1–0.5)

for _ in range(epochs):
    optimizer.zero_grad()
    probs = model(X_train)

    # base BCE loss
    bce = criterion(probs, y_train)

    # confidence in [0,1]: 0 at 0.5, 1 near 0 or 1
    confidence = torch.abs(probs - 0.5) * 2.0

    # wrong predictions: 1 if wrong, 0 if correct
    hard_preds = (probs >= 0.5).float()
    wrong = torch.abs(hard_preds - y_train)

    # extra penalty:
    #   - all wrong predictions get weighted by confidence
    #   - WRONG & very high-confidence get an extra (confidence^2) penalty
    high_conf_mask = confidence > 0.8
    if high_conf_mask.any():
        high_conf_wrong = (confidence[high_conf_mask] ** 2 * wrong[high_conf_mask]).mean()
    else:
        high_conf_wrong = torch.tensor(0.0, device=probs.device)

    conf_wrong_penalty = (confidence * wrong).mean() + high_conf_wrong

    loss = bce + lambda_penalty * conf_wrong_penalty
    loss.backward()
    optimizer.step()

print("Weights:", model[0].weight.data)
print("Bias:", model[0].bias.data)

Weights: tensor([[-0.0095, -0.0629, -0.0513, -0.0322,  0.1520, -0.3797, -0.3943,  0.4327,
          0.6446,  0.0078,  0.0310,  0.0357, -0.0345, -0.0046,  0.0712, -0.1047,
         -0.0242,  0.0694, -0.0578, -0.0574, -0.0148]])
Bias: tensor([-0.0363])


In [168]:
import numpy as np

# y_test is 0/1, y_pred is probabilities, actual_above_below already built
with torch.no_grad():
    y_pred = model(X_test)

def interpret_prediction(prob):
    if prob >= 0.80:
        return "High confidence above average"
    elif prob >= 0.65:
        return "Moderate confidence above average"
    elif prob >= 0.55:
        return "Uncertain, could be above average"
    elif prob >= 0.45:
        return "Uncertain, could be average"
    elif prob >= 0.35:
        return "Uncertain, could be below average"
    elif prob >= 0.25:
        return "Moderate confidence below average"
    elif prob <= 0.20:
        return "High confidence below average"
    else:
        return "High confidence below average"

labels = []
correct = []

for i, prob in enumerate(y_pred):
    p = prob.item()
    lbl = interpret_prediction(p)
    labels.append(lbl)

    # model's hard prediction: above if p >= 0.5
    pred_cls = 1.0 if p >= 0.5 else 0.0
    true_cls = y_test[i].item()
    correct.append(1 if pred_cls == true_cls else 0)

labels = np.array(labels)
correct = np.array(correct)

# counters for right and wrong decisions (overall)
total_predictions = len(correct)
total_correct = int(correct.sum())
total_wrong = int(total_predictions - total_correct)
print(f"\nModel decisions: {total_correct} correct, {total_wrong} wrong out of {total_predictions}")

# split counters by confidence level (high, moderate, low)
high_total = high_correct = 0
moderate_total = moderate_correct = 0
low_total = low_correct = 0

for lbl, is_corr in zip(labels, correct):
    if lbl.startswith("High confidence"):
        high_total += 1
        if is_corr:
            high_correct += 1
    elif lbl.startswith("Moderate confidence"):
        moderate_total += 1
        if is_corr:
            moderate_correct += 1
    else:  # treat everything else as low confidence / uncertain
        low_total += 1
        if is_corr:
            low_correct += 1

def _print_conf_summary(name, total, correct):
    if total == 0:
        print(f"{name}: no examples")
    else:
        wrong = total - correct
        acc = correct / total
        print(f"{name}: {correct} correct, {wrong} wrong out of {total} (acc={acc:5.2f})")

print("\nBy confidence level:")
_print_conf_summary("High confidence", high_total, high_correct)
_print_conf_summary("Moderate confidence", moderate_total, moderate_correct)
_print_conf_summary("Low confidence", low_total, low_correct)

print("\nCalibration per verbal label:")
for lbl in sorted(set(labels)):
    mask = labels == lbl
    n = mask.sum()
    acc = correct[mask].mean() if n > 0 else float("nan")
    print(f"{lbl:35s} n={n:2d}, accuracy={acc:5.2f}")


Model decisions: 11 correct, 9 wrong out of 20

By confidence level:
High confidence: no examples
Moderate confidence: 3 correct, 1 wrong out of 4 (acc= 0.75)
Low confidence: 8 correct, 8 wrong out of 16 (acc= 0.50)

Calibration per verbal label:
Moderate confidence below average   n= 4, accuracy= 0.75
Uncertain, could be average         n= 6, accuracy= 0.50
Uncertain, could be below average   n=10, accuracy= 0.50


In [169]:
# hardcoded to test today's game
X_test_np_today = [[36.6, 23, 7, 4, .522, .143, .688, .543, 1.000, 4, 5, 5, 16.5, 2, 29, 1, 0, 114.0, 119.7, -5.7, 96.9]]
X_test_today = torch.tensor(X_test_np_today, dtype=torch.float32)
y_today = model(X_test_today)

# above average vs. below average calculation
predicted_above_below = []
actual_above_below = []
for pred in y_today:
    if pred.item() > 0.5:
        predicted_above_below.append("Above Average")
    else:
        predicted_above_below.append("Below Average")

for actual in y_test:
    if actual.item() > 0.5:
        actual_above_below.append("Above Average")
    else:
        actual_above_below.append("Below Average")

print(predicted_above_below)
y_today.item()

['Below Average']


0.48507434129714966

In [170]:
interpreted_data = []
for pred in y_pred:
    interpreted_data.append(interpret_prediction(pred.item()))
    print(str(pred.item()) + " - "  + interpret_prediction(pred.item()) + " - " + str(actual_above_below[len(interpreted_data)-1]))

0.28951922059059143 - Moderate confidence below average - Above Average
0.33777040243148804 - Moderate confidence below average - Below Average
0.48631641268730164 - Uncertain, could be average - Below Average
0.46084344387054443 - Uncertain, could be average - Below Average
0.3795250356197357 - Uncertain, could be below average - Below Average
0.3901735544204712 - Uncertain, could be below average - Below Average
0.44885846972465515 - Uncertain, could be below average - Below Average
0.346571683883667 - Moderate confidence below average - Below Average
0.43041670322418213 - Uncertain, could be below average - Above Average
0.32709282636642456 - Moderate confidence below average - Below Average
0.36155256628990173 - Uncertain, could be below average - Below Average
0.3777649700641632 - Uncertain, could be below average - Above Average
0.4702097773551941 - Uncertain, could be average - Above Average
0.37064847350120544 - Uncertain, could be below average - Below Average
0.44938892126083